In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/final_project'

In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Model

from PIL import Image

In [ ]:
# Load dataset images into the DataFrame and convert into RGB (required by MobileNetV2)

dataset = []

for emotion in os.listdir('dataset'):
    if os.path.isdir(f'dataset/{emotion}'):
        for image_filename in os.listdir(f'dataset/{emotion}'):
          image = Image.open(f'dataset/{emotion}/{image_filename}').convert("RGB")
          pixels = np.array(image).astype(float)
          dataset.append((pixels, emotion))

df = pd.DataFrame(dataset, columns=['pixels', 'emotion'])

In [ ]:
# Normalizacion of the data.

X = np.array((df['pixels']/255).values.tolist())
Y = LabelBinarizer().fit_transform(df['emotion'])

In [ ]:
# Split dataset into train and test data.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Attributes for input (images width, high and channel) and output (number of labels).

_, IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS = X.shape
_, NUM_LABELS = Y.shape

In [ ]:
base_model = MobileNetV2(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS),
                         include_top=False,
                         weights='imagenet')

#base_model.trainable = False

modelMobNet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(NUM_LABELS, activation='softmax')
])

modelMobNet.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
tensorboarddata = TensorBoard(log_dir='logs/train_borad')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
modelMobNet.fit(X_train, Y_train, 
                batch_size=32, 
                epochs=25,  
                validation_data=(X_test, Y_test), 
                callbacks=[TensorBoard(log_dir="logs/modelMobNet")])


In [ ]:
score, acc = modelMobNet.evaluate(X_test, Y_test, batch_size=100)
print('Test score:', score)
print("Test accuracy:", acc)